In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, TimeDistributed
from keras.layers import Concatenate, Flatten
from keras.layers import GRU, Conv2D, MaxPooling2D
from keras.layers import Input, Reshape, Dot, Add
from keras.models import Model
from keras.optimizers import Adam
from keras.optimizers import RMSprop
# from keras.utils.vis_utils import plot_model
import keras
import keras.backend as K
from data_handle import *
from gensim_wrapper import *
from utils import *
import gensim
import random
import numpy as np
import tensorflow as tf
seed_val = 1000
random.seed(seed_val)
np.random.seed(seed_val)
tf.set_random_seed(seed_val)

c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
def evaluate(final_embedding, word2int, embed_size):
    gensim = GensimWrapper(embed_size, 0, log=False)
    gensim.set_embeddings(word2int, final_embedding)
    result = gensim.evaluate()
    for key in result:
        print("{0}: {1:.2f}%".format(key, result[key]), end=' ')
    print()

In [3]:
words = read_file()
vocab, word2int, int2word = build_vocab(words)
int_words = words_to_ints(word2int, words)
word2freq = get_frequency(words, word2int, int2word)
char2int, int2char, char2tup, tup2char, n_consonant, n_vowel = build_charset()
ns_unigrams = ns_sample(word2freq, word2int, int2word, .75)
n_chars = 11 + 2 
n_features = len(char2int)
batch_size = 120
embed_size = 128
skip_window = 5

In [16]:
def parseVec(file, delimiter):
    lines = open(file, encoding='utf8').readlines()
    vocab_size, embed_size = [int(s) for s in lines[0].split()]
    embeddings = np.ndarray((vocab_size, embed_size), dtype=np.float32)
    for i in range(vocab_size):
        line = lines[i+1].split(delimiter)[:-1]
        word = line[0]
        if word in word2int:
            wordvec = np.array([float(j) for j in line[1:]])
            embeddings[word2int[word]] = wordvec
    return embeddings

In [36]:
fast = parseVec('results/model.vec', ' ')

In [20]:
cew = parseVec('results/vec.txt', '\t')

In [37]:
utils = Utils(embedding=normalize(fast), word2int=word2int, int2word=int2char)
utils.sorted_sim("ኢትዮጵያ")

[('ኢትዮጵያ', 1.0),
 ('ኢትዮጵያበኢትዮጵያ', 0.8293742),
 ('ኢትዮጵያዋ', 0.78924227),
 ('ኢትዮጵያያን', 0.7690252),
 ('ኤርኢትዮጵያ', 0.76778996),
 ('ኢትዮጵያያ', 0.7673042),
 ('መሆንኢትዮጵያ', 0.75970054),
 ('ኢትዮጵያስ', 0.74924195),
 ('ኃይሌኢትዮጵያ', 0.7427623),
 ('ወደኢትዮጵያ', 0.7392944)]

In [22]:
utils = Utils(embedding=cew, word2int=word2int, int2word=int2char)
utils.sorted_sim("ኢትዮጵያ")

[('የሚያንፀባርቅ', 9.574177),
 ('ጸሐፊውን', 9.566519),
 ('እየገለጽን', 9.509059),
 ('አፍሪካ', 9.402455),
 ('ኢትዮጵያ', 9.243259),
 ('በኢሜይል', 9.170645),
 ('አድራሻቸው', 9.09967),
 ('በተባበሩት', 8.940824),
 ('የአፍሪካ', 8.9210205),
 ('በአፍሪካ', 8.907141)]

In [38]:
evaluate(embed_size=embed_size,final_embedding=normalize(fast), word2int=word2int)
evaluate(embed_size=embed_size,final_embedding=fast, word2int=word2int)

c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


anomaly: 75.23% semantic: 3.78% syntactic: 10.61% 
anomaly: 66.06% semantic: 7.03% syntactic: 9.85% 
